In [1]:
!pip install -U altair

    100% |████████████████████████████████| 737kB 23.2MB/s ta 0:00:01
Requirement not upgraded as not directly required: jinja2 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from altair) (2.10)
Requirement not upgraded as not directly required: numpy in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from altair) (1.14.3)
Requirement not upgraded as not directly required: toolz in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from altair) (0.9.0)
Requirement not upgraded as not directly required: entrypoints in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from altair) (0.2.3)
Requirement not upgraded as not directly required: jsonschema in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from altair) (2.6.0)
Requirement not upgraded as not directly required: pandas>=0.18 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from altair) (0.24.2)
Requirement not upgrad

# Clustering Crypto

In [26]:
# Initial imports
import altair as alt
import pandas as pd
#import hvplot.pandas
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Data Preprocessing

In [9]:
# Loade the cryptocurrencies data
file_path = Path("Resources/crypto_data.csv")
crypto_df = pd.read_csv(file_path, index_col=0)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [10]:
# Keep only cryptocurrencies that are on trading
crypto_df=crypto_df.loc[crypto_df['IsTrading'] == True]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [11]:
# Keep only cryptocurrencies with a working algorithm
crypto_df=crypto_df.loc[crypto_df['Algorithm'] != '']
crypto_df=crypto_df.loc[crypto_df['Algorithm'].notnull()]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [12]:
# Remove the "IsTrading" column
crypto_df.drop(columns='IsTrading', inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [13]:
# Remove rows with at least 1 null value
crypto_df.dropna(inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [14]:
# Remove rows with cryptocurrencies without coins mined
crypto_df = crypto_df.loc[crypto_df['TotalCoinsMined'] > 0]
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [15]:
# Fetch the cryptocurrencies names prior to drop them from crypto_df
coins_name = pd.DataFrame(crypto_df['CoinName'].copy())
coins_name.head()


,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [16]:
# Remove the cryptocurrency name since it's not going to be used on the clustering algorithm
crypto_df.drop(columns='CoinName',inplace=True)
crypto_df.head()


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [17]:
# Create dummies variables for text features
X =  pd.get_dummies(crypto_df,columns=['Algorithm','ProofType'])
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# Standardize data
X= StandardScaler().fit_transform(X)
X

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning:

Data with input dtype uint8, float64, object were all converted to float64 by StandardScaler.

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning:

Data with input dtype uint8, float64, object were all converted to float64 by StandardScaler.



array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Reducing Dimensions Using PCA

In [21]:
# Use PCA to reduce dimension to 3 principal components

pca = PCA(n_components=3)

# Get three principal components for the data.
crypt_pca = pca.fit_transform(X)

In [22]:
# Create a DataFrame with the principal components data
pcs_df = pd.DataFrame(data=crypt_pca, columns=["PC1", "PC2", "PC3"]).set_index(crypto_df.index)
pcs_df.head()


,PC1,PC2,PC3
42,-0.339288,0.982325,-0.582451
404,-0.322630,0.982359,-0.582842
1337,2.315960,1.652076,-0.677838
BTC,-0.135425,-1.246131,0.207795
ETH,-0.151837,-2.061902,0.404324


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [35]:

inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range ok k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

    


# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
alt.Chart(df_elbow).mark_line().encode(
                x='k', 
                y='inertia').properties(
                    title='Elbow Curve',
                    width=800
)



alt.Chart(...)

Running K-Means with `k=4`

In [43]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)
# Fit the model
model.fit(pcs_df)
# Predict clusters
pred_clusters = model.predict(pcs_df)
# Create a new DataFrame including predicted clusters and cryptocurrencies feature
#pcs_df .index = crypt_names.index
clustered_df = pd.concat([crypto_df, pcs_df, coins_name], axis=1)
clustered_df['Class'] = model.labels_
clustered_df.head(10)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.339288,0.982325,-0.582451,42 Coin,3
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.322630,0.982359,-0.582842,404Coin,3
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.315960,1.652076,-0.677838,EliteCoin,3
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.135425,-1.246131,0.207795,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.151837,-2.061902,0.404324,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.173665,-1.161446,-0.009943,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.377625,1.277239,-0.494436,Dash,3
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.135126,-2.170833,0.395629,Monero,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.150279,-2.062012,0.404303,Ethereum Classic,0
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.161752,-2.213536,0.453876,ZCash,0


### Visualizing Results

#### 3D-Scatter with Clusters

In [61]:
# Create a 3D-Scatter with the PCA data and the clusters
alt.Chart(clustered_df).mark_circle(size=60).encode(
                x='PC1', 
                y='PC2', 
                color='Class',
                tooltip=["CoinName", "Algorithm", 
                    "TotalCoinsMined", "TotalCoinSupply"]).interactive()


alt.Chart(...)

#### Table of Tradable Cryptocurrencies

In [81]:
# Table with tradable cryptos
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(
        clustered_df[
            [
                "CoinName",
                "Algorithm",
                "ProofType",
                "TotalCoinSupply",
                "TotalCoinsMined",
                "Class",
            ]
        ]
    )

,CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class
42,42 Coin,Scrypt,PoW/PoS,4.200000e-11,0.000000e+00,3
404,404Coin,Scrypt,PoW/PoS,5.320000e-04,1.065855e-03,3
1337,EliteCoin,X13,PoW/PoS,3.141593e-01,2.957551e-02,3
BTC,Bitcoin,SHA-256,PoW,2.100000e-05,1.810842e-05,0
ETH,Ethereum,Ethash,PoW,0.000000e+00,1.087731e-04,0
LTC,Litecoin,Scrypt,PoW,8.400000e-05,6.367669e-05,0
DASH,Dash,X11,PoW/PoS,2.200000e-05,9.122581e-06,3
XMR,Monero,CryptoNight-V7,PoW,0.000000e+00,1.737505e-05,0
ETC,Ethereum Classic,Ethash,PoW,2.100000e-04,1.145060e-04,0
ZEC,ZCash,Equihash,PoW,2.100000e-05,7.457675e-06,0


In [70]:
# Print the total number of tradable cryptocurrencies
print(f"The total number of tradeable cryptocurrencies is {len(clustered_df)}")


The total number of tradeable cryptocurrencies is 532


#### Scatter Plot with Tradable Cryptocurrencies

In [71]:
scaled_data = MinMaxScaler().fit_transform(clustered_df[['TotalCoinSupply', 'TotalCoinsMined']])
scaled_data_df = pd.DataFrame(clustered_df)
scaled_data_df['TotalCoinSupply'] = scaled_data[:,0]
scaled_data_df['TotalCoinsMined'] = scaled_data[:,1]
scaled_data_df.set_index(clustered_df.index, inplace=True)
scaled_data_df.head(10)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning:

Data with input dtype float64, object were all converted to float64 by MinMaxScaler.



,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,Class
42,Scrypt,PoW/PoS,0.000000,4.200000e-11,-0.339288,0.982325,-0.582451,42 Coin,3
404,Scrypt,PoW/PoS,0.001066,5.320000e-04,-0.322630,0.982359,-0.582842,404Coin,3
1337,X13,PoW/PoS,0.029576,3.141593e-01,2.315960,1.652076,-0.677838,EliteCoin,3
BTC,SHA-256,PoW,0.000018,2.100000e-05,-0.135425,-1.246131,0.207795,Bitcoin,0
ETH,Ethash,PoW,0.000109,0.000000e+00,-0.151837,-2.061902,0.404324,Ethereum,0
LTC,Scrypt,PoW,0.000064,8.400000e-05,-0.173665,-1.161446,-0.009943,Litecoin,0
DASH,X11,PoW/PoS,0.000009,2.200000e-05,-0.377625,1.277239,-0.494436,Dash,3
XMR,CryptoNight-V7,PoW,0.000017,0.000000e+00,-0.135126,-2.170833,0.395629,Monero,0
ETC,Ethash,PoW,0.000115,2.100000e-04,-0.150279,-2.062012,0.404303,Ethereum Classic,0
ZEC,Equihash,PoW,0.000007,2.100000e-05,-0.161752,-2.213536,0.453876,ZCash,0


In [80]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
alt.Chart(scaled_data_df).mark_circle(size=60).encode(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    color='Class',
).interactive()

alt.Chart(...)